In [3]:
import os
import numpy as np
import pandas as pd
import altair as alt
import sys
import json

sys.path.insert(0, '..')
from agents import PsAgent
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [70]:
path = '../data/models/colision_0005'

In [71]:
lista = [i for i in os.listdir(f'{path}') if i not in ['.ipynb_checkpoints']]

In [72]:
lista

['2025-03-27_13-20-00.582638__24',
 '2025-03-27_13-20-00.573355__8',
 '2025-03-27_13-20-00.578195__16',
 '2025-03-27_13-20-00.580019__19',
 '2025-03-27_13-20-00.571222__3',
 '2025-03-27_13-20-00.583871__26',
 '2025-03-27_13-20-00.574117__9',
 '2025-03-27_13-20-00.581868__23',
 '2025-03-27_13-20-00.584793__27',
 '2025-03-27_13-20-00.580538__20',
 '2025-03-27_13-20-00.572436__6',
 '2025-03-27_13-20-00.594601__30',
 '2025-03-27_13-20-00.576201__12',
 '2025-03-27_13-20-00.577568__15',
 '2025-03-27_13-20-00.578581__17',
 '2025-03-27_13-20-00.572902__7',
 '2025-03-27_13-20-00.580900__21',
 '2025-03-27_13-20-00.577195__14',
 '2025-03-27_13-20-00.575563__11',
 '2025-03-27_13-20-00.586024__29',
 '2025-03-27_13-20-00.571708__4',
 '2025-03-27_13-20-00.581337__22',
 '2025-03-27_13-20-00.574631__10',
 '2025-03-27_13-20-00.570391__1',
 '2025-03-27_13-20-00.579724__18',
 '2025-03-27_13-20-00.572064__5',
 '2025-03-27_13-20-00.583286__25',
 '2025-03-27_13-20-00.576797__13',
 '2025-03-27_13-20-00.569997

In [84]:
dados = []
for model in lista:    
    with open(f'{path}/{model}/args.json', 'r') as file:
        args = json.load(file)
        
    n_reflect = args['num_reflections']
    tao = args['tao']
    colision_flag = args['colision']
    agent = PsAgent.load(f'{path}/{model}')
    matrix = agent.h_matrix
    if colision_flag:
        for state in [0,1]:
            for colision in [0,1]:
                for T in range(0,300):
                    observation = [state,T,colision]
                    percept = agent.percept_preprocess(observation)

                    h_vector = matrix[:, percept]
                    sum_ = np.sum(h_vector)
                    probs = h_vector/sum_

                    nao_troca = probs[0]
                    troca = probs[1]

                    _ = {
                        'model':model,
                        'reflections':args['num_reflections'],
                        'T':T/tao,
                        'state':state,
                        'colision_flag':colision_flag,
                        'colision':colision,
                        'troca':troca,
                        'nao_troca':nao_troca,
                    }

                    dados.append(_)
    else:
        for state in [0,1]:
            for T in range(0,300):
                observation = [state,T]
                percept = agent.percept_preprocess(observation)

                h_vector = matrix[:, percept]
                sum_ = np.sum(h_vector)
                probs = h_vector/sum_

                nao_troca = probs[0]
                troca = probs[1]

                _ = {
                    'model':model,
                    'reflections':args['num_reflections'],
                    'T':T/tao,
                    'state':state,
                    'colision_flag':colision_flag,
                    'colision':np.nan,
                    'troca':troca,
                    'nao_troca':nao_troca,
                }

                dados.append(_)

# Sem colisão

In [85]:
db = pd.DataFrame(dados)
db

,model,reflections,T,state,colision_flag,colision,troca,nao_troca
0,2025-03-27_13-20-00.582638__24,0,0.0000,0,1,0,0.001213,0.998787
1,2025-03-27_13-20-00.582638__24,0,0.0001,0,1,0,0.005566,0.994434
2,2025-03-27_13-20-00.582638__24,0,0.0002,0,1,0,0.001133,0.998867
3,2025-03-27_13-20-00.582638__24,0,0.0003,0,1,0,0.007961,0.992039
4,2025-03-27_13-20-00.582638__24,0,0.0004,0,1,0,0.023910,0.976090
...,...,...,...,...,...,...,...,...
38395,2025-03-27_13-20-00.585512__28,0,0.0295,1,1,1,0.000811,0.999189
38396,2025-03-27_13-20-00.585512__28,0,0.0296,1,1,1,0.000779,0.999221
38397,2025-03-27_13-20-00.585512__28,0,0.0297,1,1,1,0.000835,0.999165
38398,2025-03-27_13-20-00.585512__28,0,0.0298,1,1,1,0.000802,0.999198


In [86]:
db = pd.DataFrame(dados)
#db = db[~db['colision_flag'].apply(bool)]
db = db.sort_values(['model','state','T']).reset_index(drop = True)

db['prod_nao_troca'] = db.groupby(['model','state'])['nao_troca'].cumprod().shift(1)
db['P_T_troca'] = db['prod_nao_troca']*db['troca']
db['sum_P_T_troca'] = db.groupby(['model','state'])['P_T_troca'].cumsum().shift(1)

db_mean = db.groupby(['reflections','T','state'])[
    ['troca', 'P_T_troca', 'sum_P_T_troca']
].mean().reset_index()

db_mean.head()

,reflections,T,state,troca,P_T_troca,sum_P_T_troca
0,0,0.0000,0,0.002858,0.001985,0.495668
1,0,0.0000,1,0.000930,0.000418,0.497171
2,0,0.0001,0,0.004281,0.004245,0.005866
3,0,0.0001,1,0.002645,0.002641,0.002618
4,0,0.0002,0,0.004624,0.004553,0.014322


In [87]:
alt.Chart(db_mean).mark_line().encode(
    x = 'T',
    y = alt.Y('troca').axis(format = '%'),
    row = 'state:N',
    color = 'reflections:N'
).properties(height = 100).resolve_scale(y = 'independent')

alt.Chart(...)

In [88]:
alt.Chart(db_mean).mark_line().encode(
    x = 'T',
    y = alt.Y('P_T_troca').axis(format = '%'),
    row = 'state:N',
    color = 'reflections:N'
).properties(height = 100).resolve_scale(y = 'independent')

alt.Chart(...)

In [89]:
alt.Chart(db_mean).mark_line().encode(
    x = 'T',
    y = alt.Y('sum_P_T_troca').axis(format = '%'),
    color = 'state:N',
).properties(height = 100).resolve_scale(y = 'independent')

alt.Chart(...)

# Com colisão

In [90]:
db = pd.DataFrame(dados)

db = db[db['colision_flag'].apply(bool)]
db = db.sort_values(['model','state','colision','T']).reset_index(drop = True)
db['prod_nao_troca'] = db.groupby(['model','state','colision'])['nao_troca'].cumprod().shift(1)
db['P_T_troca'] = db['prod_nao_troca']*db['troca']
db['sum_P_T_troca'] = db.groupby(['model','state','colision'])['P_T_troca'].cumsum().shift(1)

db_mean = db[db['T'] > 2/tao].groupby(['reflections','T','state','colision'])[
    ['troca', 'P_T_troca', 'sum_P_T_troca']
].mean().reset_index()

db_mean.head()

,reflections,T,state,colision,troca,P_T_troca,sum_P_T_troca
0,0,0.0003,0,0,0.002976,0.002959,0.008035
1,0,0.0003,0,1,0.004916,0.004848,0.010142
2,0,0.0003,1,0,0.000393,0.000392,0.003810
3,0,0.0003,1,1,0.001241,0.001237,0.004095
4,0,0.0004,0,0,0.003466,0.003417,0.010994


In [91]:
alt.Chart(db_mean).mark_line().encode(
    x = 'T',
    y = alt.Y('troca').axis(format = '%'),
    row = 'state:N',
    column = 'colision:N',
    color = 'reflections:N'
).properties(height = 100).resolve_scale(y = 'independent')

alt.Chart(...)

In [92]:
alt.Chart(db_mean).mark_line().encode(
    x = 'T',
    y = alt.Y('P_T_troca').axis(format = '%'),
    row = 'state:N',
    column = 'colision:N',
    color = 'reflections:N'
).properties(height = 100).resolve_scale(y = 'independent')

alt.Chart(...)

In [93]:
alt.Chart(db_mean).mark_line().encode(
    x = 'T',
    y = alt.Y('sum_P_T_troca').axis(format = '%'),
    color = 'state:N',
    row = 'colision:N',
).properties(height = 100).resolve_scale(y = 'independent')

alt.Chart(...)

In [94]:
 alt.Chart(db_mean).mark_line().encode(
    x = 'T',
    y = alt.Y('sum_P_T_troca').axis(format = '%'),
    row = 'state:N',
    color = 'colision:N',
).properties(height = 100).resolve_scale(y = 'independent')

alt.Chart(...)